In [1]:
# we are using the ipympl backend
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# for paper-ready export
plt.style.use("v_arial")
import matplotlib
matplotlib.rc('pdf', fonttype=42)

In [2]:
from ipywidgets import interact

In [3]:
import matplotlib.gridspec as gs

In [4]:
import deepdish as dd

In [5]:
folder = Path(r"J:\_Shared\stytra\phototaxis")

In [6]:
from utilities import Experiment, extract_bouts

In [7]:
experiments = []
for i, expfile in enumerate(folder.glob("**/*_metadata.json")):
    exp = Experiment(expfile)
    bouts, cont = extract_bouts(exp)
    print(i, exp.path, len(bouts))
    exp.bouts = bouts
    experiments.append(exp)

0 J:\_Shared\stytra\phototaxis\181127_f6 420
1 J:\_Shared\stytra\phototaxis\181128_f10 440
2 J:\_Shared\stytra\phototaxis\181128_f11 504
3 J:\_Shared\stytra\phototaxis\181128_f12 604
4 J:\_Shared\stytra\phototaxis\181128_f13 1063
5 J:\_Shared\stytra\phototaxis\181128_f15 435
6 J:\_Shared\stytra\phototaxis\181128_f7 794
7 J:\_Shared\stytra\phototaxis\181128_f8 572
8 J:\_Shared\stytra\phototaxis\181128_f9 909
9 J:\_Shared\stytra\phototaxis\181129_f2 709


## Plot bouts from an experiment

In [8]:
fig, ax = plt.subplots()
for b in experiments[0].bouts:
    ax.plot(b.x, b.y)
ax.set_aspect(1)

FigureCanvasNbAgg()

# Analyse turning stats

In [9]:
from utilities import reduce_to_pi

In [10]:
def bout_angles(bouts, t_stim, stim_status):
    thetas = []
    for bout_df in bouts:
        # if the bout is not within the phototaxis stimulus, do not include it
        if np.interp(bout_df["t"].iloc[0], t_stim, stim_status) == 0:
            continue
        th = bout_df["theta"].values
        if not np.any(np.diff(th)>0.2):
            thetas.append(np.nanmean(th[:10])-np.nanmean(th[:-10]))
    thetas = reduce_to_pi(np.array(thetas))
    thetas = thetas[~np.isnan(thetas)]
    return thetas

In [11]:
for exp in experiments:
    # make an array to determine the phototaxis on and off times
    t_stim = np.empty(len(exp["stimulus"]["log"])*2)
    stim_status = np.empty(len(t_stim), np.bool)
    for i, item in enumerate(exp["stimulus"]["log"]):
        t_stim[i*2] = item["t_start"]
        t_stim[i*2+1] = item["t_stop"]
        stim_status[i*2:(i+1)*2]= item["name"] != "flash"
    exp.angles = bout_angles(exp.bouts, t_stim, stim_status)

C:\Users\vilim\anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in greater
  
C:\Users\vilim\anaconda3\lib\site-packages\ipykernel_launcher.py:9: RuntimeWarning: Mean of empty slice
  if __name__ == '__main__':
C:\Users\vilim\Analysis\BehaviouralAnalysis\example_stytra_analysis\utilities.py:330: RuntimeWarning: invalid value encountered in remainder
  return np.mod(ar + np.pi, np.pi * 2) - np.pi


In [12]:
hist_range = (-150,200)

## Bin the bouts by angle

In [13]:
bcs = []
hist_bins = np.linspace(*hist_range, 91)
for exp in experiments:
    # the angles are negative to compare with Huang et al.
    bin_counts, _ = np.histogram(-exp.angles*180/np.pi, hist_bins, density=True)
    bcs.append(bin_counts)

In [15]:
full_counts = np.stack(bcs, 0)

In [17]:
plt.figure(figsize=(6,4))
plt.plot((hist_bins[:-1]+hist_bins[1:])/2, np.nanmean(full_counts,0)*100)
plt.xlabel("bout angle")
plt.tight_layout()
plt.xlim(-150,200)
plt.savefig("Plots/phtotoaxis_angles.pdf")

FigureCanvasNbAgg()

## Browse individual experiments

In [18]:
fig, ax = plt.subplots()
@interact
def show_plot(i:(0, len(experiments)-1)):
    ax.clear()
    exp = experiments[i]
    ax.hist(-exp.angles*180/np.pi, bins=122, histtype="step", range=(-150, 200), density=True);
    plt.xlim(-150,200)
    plt.title(str(exp.path))

FigureCanvasNbAgg()

interactive(children=(IntSlider(value=4, description='i', max=9), Output()), _dom_classes=('widget-interact',)…